In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize 
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 11.0 GB  I Proc size: 137.7 MB
GPU RAM Free: 11260MB | Used: 181MB | Util   2% | Total 11441MB


In [0]:
import os

In [3]:
ls

drive/  sample_data/


In [4]:
from google.colab import drive
drive.mount('/content/drive')
#4/XwCmdKiWvbvVtpA_jh7vPaL1kqdreQE7s6dR7oocs6Jzb0GsuiaxJ5w

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ls

drive/  sample_data/


In [6]:
cd drive

/content/drive


In [7]:
ls

'My Drive'/


In [8]:
cd My Drive

/content/drive/My Drive


In [9]:
ls

 1stone.h5
 20151122_223055.mp4
'AAA-Literature reviews:.gdoc'
 AAARBI.gdoc
'AAA SOP Progress.gdoc'
'AAA stopwords.gdoc'
'AA BAV Project.gdoc'
'Aa bavv2.gdoc'
 ABC.xlsx
 ans.csv
 ASD.gdoc
 bidirectional42i.h5
 bidirectionalffhai.h5
 bidirectional.h5
 bidirectionalhai.h5
 bidirectionallstmwithpreprocessing.h5
 bidirectionalwithoutglobal.h5
 CNN_Data/
'Colab Notebooks'/
 companies.docx
'Copy of glove.twitter.27B.100d.txt'
'Copy of glove.twitter.27B.50d.txt'
 datarnn.csv
 DESCRIPTION.docx
 DESCRIPTION.gdoc
'dna structure1.ppt'
'DRM Ass 2.2.xlsx'
'DRM Ass 2.2.xlsx.gsheet'
 dt/
 GAIL_Group37_Futures.xlsx
 GAIL_Group37_Futures.xlsx.gsheet
'GAIL_Group37_Underlying Asset.xlsx'
'GAIL_Group37_Underlying Asset.xlsx.gsheet'
 GANs.zip
 GroceryProcessed.csv
 Hatespeech-data/
'Ian Goodfellow, Yoshua Bengio, Aaron Courville-Deep Learning-MIT Press (2016).pdf'
'Ideas and constrains of the strategies.gdoc'
 inc_occ_gender.csv
 instructions_1.pdf
'log returns.xlsx'
 lstmheading.csv
 marketdata_sample.csv

In [10]:
cd Hatespeech-data

/content/drive/My Drive/Hatespeech-data


In [11]:
ls

bidirectionallstmwithoutpreprocess.h5  Codes/  test.csv  train.csv


In [0]:
datapathtrain=(os.getcwd())+"/train.csv"
datapathtest=(os.getcwd())+"/test.csv"

In [0]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, Embedding
from keras.layers import Bidirectional, GlobalMaxPool1D,MaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, Input

In [0]:
from sklearn.utils import shuffle
train=(pd.read_csv(datapathtrain,encoding='latin-1'))
test=(pd.read_csv(datapathtest,encoding='latin-1'))

In [15]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [16]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [17]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [18]:
test.tail()

,id,comment_text
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."
153163,ffffce3fb183ee80,""" \n :::Stop already. Your bullshit is not wel..."


In [0]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [0]:
sentences_train = train["comment_text"].fillna("na").values

In [0]:
y = train[classes].values

In [22]:
y[0]

array([0, 0, 0, 0, 0, 0])

In [0]:
sentences_test = test["comment_text"].fillna("na").values

In [0]:
max_features = 10000
maxlen = 200


In [0]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(sentences_train))
tokenized_train = tokenizer.texts_to_sequences(sentences_train)
tokenized_test = tokenizer.texts_to_sequences(sentences_test)
X_train = pad_sequences(tokenized_train, maxlen=maxlen)
X_test = pad_sequences(tokenized_test, maxlen=maxlen)

In [26]:
print(X_train.shape)

(159571, 200)


In [27]:
print(y.shape)

(159571, 6)


In [0]:
inp = Input(shape=(maxlen, ))

In [0]:
x = Embedding(max_features, 128)(inp)

In [0]:
x = Bidirectional(LSTM(64, recurrent_dropout=0.2, return_sequences=True))(x)

In [0]:
x = MaxPool1D(pool_size=4)(x)

In [0]:
x = Bidirectional(LSTM(64, recurrent_dropout=0.2, return_sequences=True))(x)

In [0]:
x = GlobalMaxPool1D()(x)

In [0]:
x = Dropout(0.2)(x)

In [0]:
#x = Bidirectional(LSTM(16, recurrent_dropout=0.2, return_sequences=True))(x)

In [0]:
#x = Dropout(0.2)(x)

In [0]:
x = Dense(64, activation="relu")(x)

In [0]:
x = Dropout(0.1)(x)

In [0]:
x = Dense(6, activation="sigmoid")(x)


In [0]:
model = Model(inputs=inp, outputs=x)

In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
checkpoint = ModelCheckpoint('bidirectionallstmgg.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [0]:
early = EarlyStopping(monitor="val_loss", mode="auto", patience=3)

In [0]:
callbacks_list = [checkpoint, early] 

In [0]:
epochs=5
batch_size=1024

In [72]:
model.fit(X_train, y, batch_size=batch_size,epochs=epochs, validation_split=0.2, callbacks=callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/5
127656/127656 [==============================] - 200s 2ms/step - loss: 0.2086 - acc: 0.9354 - val_loss: 0.1211 - val_acc: 0.9637

Epoch 00001: val_loss improved from inf to 0.12114, saving model to bidirectionallstmgg.h5
Epoch 2/5
127656/127656 [==============================] - 194s 2ms/step - loss: 0.0885 - acc: 0.9721 - val_loss: 0.0615 - val_acc: 0.9780

Epoch 00002: val_loss improved from 0.12114 to 0.06150, saving model to bidirectionallstmgg.h5
Epoch 3/5
127656/127656 [==============================] - 194s 2ms/step - loss: 0.0567 - acc: 0.9796 - val_loss: 0.0542 - val_acc: 0.9807

Epoch 00003: val_loss improved from 0.06150 to 0.05419, saving model to bidirectionallstmgg.h5
Epoch 4/5
127656/127656 [==============================] - 194s 2ms/step - loss: 0.0520 - acc: 0.9807 - val_loss: 0.0534 - val_acc: 0.9811

Epoch 00004: val_loss improved from 0.05419 to 0.05335, saving model to bidirectionallstmgg.h5
Epoch 5/5
127